import plaidml.keras
plaidml.keras.install_backend()""

In [ ]:
import tensorflow as tf
import os
import time
import numpy as np
import PIL
import glob
import pathlib
import numpy as np
import os
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D,BatchNormalization,Activation,GRU
from tensorflow.keras.applications import EfficientNetB7, VGG16 

from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,Conv1D, MaxPooling1D,
    MaxPooling2D) 
from collections import deque
import sys


In [ ]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:

_dir = './toanalyze'

data_dir = pathlib.Path(_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

batch_size = 2
img_height = 274
img_width = 274
n_class=4

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        data_dir,
        subset='training',
        target_size=(img_height,img_width),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(
        data_dir,
        subset='validation',
        target_size=(img_height,img_width),
        batch_size=batch_size,
    color_mode='rgb',
        class_mode='categorical')

In [ ]:
epochs = 50
train_samples = 174     #610 
validation_samples = 42     #195
n_channel = 3

In [ ]:
#inputs, labels= training_iterator.get_next()

In [ ]:
input_shape=(img_height,img_width,n_channel)

def build_convnet_2D():
    model = Sequential()
    model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Dropout(0.5))
    model.add(Flatten())q
    model.add(Dense(32))
    
    model.add(Dense(n_class, activation='softmax'))

    return model

def build_convnet_2_2D():
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(Conv2D(256, (2,2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dropout(0.5))
    model.add(Dense(n_class, activation='softmax'))

    return model

 
def build_cnn_lstm_2D():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape)))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))

    model = Sequential()
    model.add(LSTM(8, return_sequences=True, dropout=0.5))
    # model.add(Dropout(0.2)) #added
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    return model
 
def cv2d():    
    model = Sequential()
        # 1st layer group
    model.add(Conv2D(64, (3, 3), activation='relu',
                          name='conv1',
                         padding='same',
                         input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1),
                               padding='same', name='pool1'))
        # 2nd layer group
    model.add(Conv2D(128, (3, 3), activation='relu',
                         padding='same', name='conv2',
                         ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1),
                               padding='same', name='pool2'))
        # 3rd layer group
    model.add(Conv2D(256, (3, 3), activation='relu',
                         padding='same', name='conv3a',
                         ))
    model.add(Conv2D(256, (3, 3), activation='relu',
                         padding='same', name='conv3b',
                         ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1),
                               padding='same', name='pool3'))
       

        # FC layers group

    model.add(Dense(1024, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    
    return model

def build_gru_conv2D_1():
    cmodel = Sequential()

    cmodel.add(Conv1D(128, 3, activation='relu', padding='same'))
    cmodel.add(Conv1D(64, 3, activation='relu', padding='same'))
    cmodel.add(Conv1D(32, 3, activation='relu', padding='same'))
    cmodel.add(MaxPooling1D(pool_size=2, strides=1))
    
    cmodel.add(Flatten())
    cmodel.add(Dense(32))

    
    model = Sequential()
    #model.add(TimeDistributed(cmodel))
    model.add(TimeDistributed(cmodel, input_shape=(input_shape)))
    # here, you can also use GRU or LSTM
    model.add(GRU(16))
    # and finally, we make a decision network
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5))
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    return model


def build_gru_conv2D():
    model = Sequential()# input, with 64 convolutions for 5 images
    # that have (224, 224, 3) shape
    model.add(
        TimeDistributed(
            Conv2D(64, (3,3), 
                padding='same', strides=(2,2), activation='relu'),
            
            input_shape = input_shape
        )
    )
    model.add(
        TimeDistributed( 
            Conv2D(64, (3,3), 
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed(
            MaxPooling2D((2,2), strides=(2,2))
        )
    )# Second conv, 128
    model.add(
        TimeDistributed(
            Conv2D(128, (3,3),
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed( 
            Conv2D(128, (3,3),
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed(
            MaxPooling2D((2,2), strides=(2,2))
        )
    )## and so on with 512, 1024... 
    ## ...# then we can use Flatten to reduce dimension to 1
    model.add(Flatten())## and then... merge the entire outputs to
    ## be able to use Dense(), and make predictions...
    model.add(Dense(n_class, activation='softmax'))
    return model

def simple_gru_conv2d():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape)))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(GRU(8, return_sequences=True, dropout=0.5))
    # model.add(Dropout(0.2)) #added
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    return model

def hybrid():
    model = Sequential()

    model.add(Conv1D(32, 4, activation='relu', padding='same', input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(16, 8, activation="relu", padding='same'))
    model.add(LSTM(64, return_sequences=True))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(16, 8, activation="relu", padding='same'))
    model.add(LSTM(128))
    model.add(Dense(n_class, activation='sigmoid'))
    return model


In [ ]:
optimizer = Adam(lr=1e-5, decay=1e-6)
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(verbose=1),
]

In [ ]:
# model = build_convnet_2D()
# model = build_convnet_2_2D()
# model = build_cnn_lstm_2D()
# model = build_gru_conv2D_1()
# model = cv2d()
# model = build_gru_conv2D()
# model = simple_gru_conv2d()
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# model = build_convnet_1()
# model = build_convnet_2()
# model = build_gru_conv() 
# model = cv3d()
# model = c3d()
# model = simple_gru_conv()
# model = build_cnn_lstm()
# model = hybrid()
# model = build_cnn_lstm_2D()
model = build_gru_conv2D_1()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
h = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

In [ ]:
model = build_convnet_2D()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
h2 = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

In [ ]:
model = EfficientNetB7(include_top=False, input_shape=input_shape)
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(n_class, activation='softmax')(class1)
# define new 4
model = tf.keras.Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
h3 = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

In [ ]:
model.save("VGG.h5")

In [ ]:
from tensorflow.keras.applications import NASNetLarge

In [ ]:
model = EfficientNetB7(include_top=False, input_shape=input_shape)
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(n_class, activation='softmax')(class1)
# define new 4
model = tf.keras.Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
h2 = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = h.history['accuracy']
val_acc = h.history['val_accuracy']

loss = h.history['loss']
val_loss = h.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks= np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max()/2.

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,i, format(cm[i,j], fmt), horizontalalignment='center', color='white' if cm[i,j]> thresh else 'black')

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()




In [ ]:
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
pred = model.predict(validation_generator,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
trueclasses = validation_generator.classes
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
y_pred = [labels[k] for k in predicted_class_indices]
y_true = [labels[k] for k in trueclasses]
f1_score(y_true, y_pred, average='weighted') #None
confusion_matrix(y_true, y_pred)


In [ ]:
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, list(range(4)))

In [ ]:
cm = 

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, list(range(4)),normalize=True)

In [ ]:
import imageio 
import os
import matplotlib.pyplot as plt

In [ ]:
dir = r'.\input-gray\val\Alanya\2019-12-23\ANT191223070003.PPINW4L.png'

In [ ]:

i0 = imageio.imread(dir)
plt.imshow(i0)

In [ ]:
class_info = ['west','east','center','Sparse']

class_info2 = ['Aksu','Alanya','Antalya','Finike','Gazipasa','Kas','Kemer','Kumluca','Manavgat','Serik','Sparse']

from tweaked_ImageGenerator_v2 import ImageDataGenerator
datagen = ImageDataGenerator()
train_generator=datagen.flow_from_directory(
            'newinput_charts/train', 
            target_size=(112,112), 
            class_mode='categorical',
            classes=class_info,
            color_mode='rgb', 
            batch_size=8, frames_per_step=5, shuffle=True)
validation_generator = datagen.flow_from_directory(
            directory='newinput_charts/val',
            color_mode='rgb',
            target_size=(112,112),
            classes=class_info,
            frames_per_step=5, 
            class_mode='categorical')

In [ ]:
i0.size

In [ ]:
def build_simple_conv2d():
    model = Sequential()
    model.add(Conv3D(32, (3,3, 3), input_shape=(13,224,224,1), activation='relu'))
    
    model.add(MaxPooling3D(pool_size=(2, 2,2)))

    model.add(Conv3D(32, (3, 3,3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2,2)))



    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    return model


# model = build_simple_conv2d()

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# epochs = 30
# train_samples = 1853  #1485
# validation_samples = 338 #368
# batch_size=16
# x = model.fit(
#         train_generator,
#         steps_per_epoch=train_samples // batch_size,
#         epochs=epochs,
#         validation_data=validation_generator,
#         validation_steps=validation_samples // batch_size)

In [ ]:
input_shape= (5,112,112,3)
n_class=5
def build_convnet_2():
    model = Sequential()
    model.add(Conv3D(32, (3,3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    model.add(Conv3D(64, (3,3,3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    model.add(Conv3D(128, (3,3,3), activation='relu'))
    model.add(Conv3D(128, (3,3,3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    model.add(Conv3D(256, (2,2,2), activation='relu'))
    model.add(Conv3D(256, (2,2,2), activation='relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dropout(0.5))
    model.add(Dense(1024))
    model.add(Dropout(0.5))
    model.add(Dense(n_class, activation='softmax'))

    return model

 

def build_cnn_lstm():
    cnn_model = Sequential()
    cnn_model.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

    cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
    cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

    cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
    cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(Flatten())

    model = Sequential()
    model.add(TimeDistributed(cnn_model))
    model.add(LSTM(8, return_sequences=True, dropout=0.5))
    # model.add(Dropout(0.2)) #added
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    return model


def cv3d():    
    model = Sequential()
        # 1st layer group
    model.add(Conv3D(64, (3, 3, 3), activation='relu',
                          name='conv1',
                         padding='same',
                         input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               padding='same', name='pool1'))
        # 2nd layer group
    model.add(Conv3D(128, (3, 3, 3), activation='relu',
                         padding='same', name='conv2',
                         ))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='same', name='pool2'))
        # 3rd layer group
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                         padding='same', name='conv3a',
                         ))
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                         padding='same', name='conv3b',
                         ))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='same', name='pool3'))
        # 4th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                         padding='same', name='conv4a',
                         ))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                         padding='same', name='conv4b',
                         ))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='same', name='pool4'))

        # 5th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                         padding='same', name='conv5a',
                          ))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                         padding='same', name='conv5b',
                          ))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='same', name='pool5'))
    model.add(Flatten())

        # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(n_class, activation='softmax'))
    
    return model

def c3d():
    model = Sequential()

    model.add(Conv3D(32, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(MaxPooling3D(pool_size=(2,2,2)))

     

    #Flatten Layers
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    #softmax layer
    model.add(Dense(n_class, activation='softmax'))
    return model

def build_gru_conv2():
    cmodel = Sequential()
    cmodel.add(Conv2D(128, (3,3), activation='relu', padding='same', input_shape=input_shape))
    cmodel.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    cmodel.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    cmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    
    cmodel.add(Flatten())
    cmodel.add(Dense(32))

    
    model = Sequential()
    #model.add(TimeDistributed(cmodel))
    model.add(TimeDistributed(cmodel, input_shape=input_shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(16))
    # and finally, we make a decision network
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5))
    model.add(Flatten())
    model.add(Dense(n_class, activation='softmax'))
    return model



def build_gru_conv():
    model = Sequential()# input, with 64 convolutions for 5 images
    # that have (224, 224, 3) shape
    model.add(
        TimeDistributed(
            Conv2D(64, (3,3), 
                padding='same', strides=(2,2), activation='relu'),
            
            input_shape = input_shape
        )
    )
    model.add(
        TimeDistributed( 
            Conv2D(64, (3,3), 
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed(
            MaxPooling2D((2,2), strides=(2,2))
        )
    )# Second conv, 128
    model.add(
        TimeDistributed(
            Conv2D(128, (3,3),
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed( 
            Conv2D(128, (3,3),
                padding='same', strides=(2,2), activation='relu')
        )
    )
    model.add(
        TimeDistributed(
            MaxPooling2D((2,2), strides=(2,2))
        )
    )## and so on with 512, 1024... 
    ## ...# then we can use Flatten to reduce dimension to 1
    model.add(Flatten())## and then... merge the entire outputs to
    ## be able to use Dense(), and make predictions...
    model.add(Dense(n_class, activation='softmax'))
    return model





In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np

def plot_confusion_matrix(cm, classes,normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    
       
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normalized confusion matrix')
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks= np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max()/2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i, format(cm[i,j], fmt), horizontalalignment='center', color='white' if cm[i,j]> thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


